## Instructions ##
1. Map top 10% snps to genes
2. Use additive model 


In [1]:
import pandas as pd 
import numpy as np
import bisect as bs
from sklearn import linear_model
from sklearn.model_selection import train_test_split
import pandas as pd
import scipy.stats as stat

In [2]:
SNP_url = "./1011geneSNP.csv"
Gene_url = "./GCF_000146045.2_R64_genomic.gff"

snp_yeast_matrix = pd.read_csv(SNP_url, sep = ',' , error_bad_lines=False)
del snp_yeast_matrix['#CHROM']
del snp_yeast_matrix['POS']
del snp_yeast_matrix['REF']
del snp_yeast_matrix['ALT']
del snp_yeast_matrix['ANN[*].GENE']
del snp_yeast_matrix['ANN[*].GENEID']
index_list = snp_yeast_matrix["ID"].tolist()
del snp_yeast_matrix['ID']
snp_yeast_matrix = snp_yeast_matrix.transpose()
column_names = list(snp_yeast_matrix.index)
to_delete = ["ABC", "ABF", "ASP", "BGS","BGF", "BHL","BHQ","BID","BIR","BKG","BKN","CFH","SACE_YAL","SACE_YBA",
             "SACE_YBB","SACE_YBM","SACE_YBN","SACE_YBO","SACE_YBP","SACE_YCJ","SACE_YCS","SACE_YCT","SACE_YCU","SACE_YCV","SACE_YCW",
             "SACE_YCX","SACE_YCY","SACE_YCZ","SACE_YDA","SACE_YDB","SACE_YDC","SACE_YDD", "SACE_YDE","SACE_YDF","SACE_YDG","SACE_YDH","SACE_YDI",
             "SACE_YDJ", "SACE_YDK", "SACE_YDL"]
print(len(to_delete))
index = 0
for index in range(len(to_delete)):
    curr = to_delete[index]
    curr = curr+"_"+curr
    to_delete[index] = curr
    index+=1
to_delete_index = []
for item in to_delete:
    curr_index = column_names.index(item)
    to_delete_index.append(curr_index)
snp_yeast_matrix = snp_yeast_matrix.drop(to_delete)
snp_yeast_matrix.head()

40


,0,1,2,3,4,5,6,7,8,9,...,18921,18922,18923,18924,18925,18926,18927,18928,18929,18930
AAA_AAA,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAB_AAB,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,1,0,0,0
AAC_AAC,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AAD_AAD,0,0,0,0,1,1,1,0,0,0,...,0,0,1,1,1,1,0,0,1,1
AAE_AAE,0,0,0,1,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [3]:
pheno_url = "./phenoMatrix_35ConditionsNormalizedByYPD.csv"
pheno_yeast_matrix = pd.read_csv(pheno_url, delimiter='\t', error_bad_lines=False)
YPD6AU = pheno_yeast_matrix["YPD6AU"]
YPDBENOMYL200 = pheno_yeast_matrix["YPDBENOMYL200"]
YPDNACL1M = pheno_yeast_matrix["YPDNACL1M"]
YPDSDS = pheno_yeast_matrix["YPDSDS"]
YPGALACTOSE = pheno_yeast_matrix["YPGALACTOSE"]
YPGLYCEROL = pheno_yeast_matrix["YPGLYCEROL"]
YPRIBOSE = pheno_yeast_matrix["YPRIBOSE"]
YPSORBITOL = pheno_yeast_matrix["YPSORBITOL"]
total_run = 100

## Calculate index of top associated snps in 100 runs ##

In [4]:
sum = 0
weight_vector_1 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPD6AU, test_size=0.2)
    lr1 = linear_model.LinearRegression()
    lr1.fit(x_train, y_train)
    temp = np.array(lr1.coef_)
    weight_vector_1 = [x + y for x, y in zip(weight_vector_1, temp)]
    predictions_1 = lr1.predict(x_test)
    pcc_1 = stat.pearsonr(predictions_1, y_test)
    counter+=1
    sum += pcc_1[0]

avg_assoc_1 = sum/total_run
avg_weight_vector_1 = np.divide(weight_vector_1, total_run)
print(avg_assoc_1)
print(avg_weight_vector_1)
top2000_index = (-avg_weight_vector_1).argsort()[:2000]
print (top2000_index)
print (avg_weight_vector_1[top2000_index])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_2 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDBENOMYL200, test_size=0.2)
    lr2 = linear_model.LinearRegression()
    lr2.fit(x_train, y_train)
    temp = np.array(lr2.coef_)
    weight_vector_2 = [x + y for x, y in zip(weight_vector_2, temp)]
    predictions_2 = lr2.predict(x_test)
    pcc_2 = stat.pearsonr(predictions_2, y_test)
    counter+=1
    sum += pcc_2[0]

avg_assoc_2 = sum/total_run
avg_weight_vector_2 = np.divide(weight_vector_2, total_run)
print(avg_assoc_2)
print(avg_weight_vector_2)
top2000_index_2 = (-avg_weight_vector_2).argsort()[:2000]
print (top2000_index_2)
print (avg_weight_vector_2[top2000_index_2])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_3 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDNACL1M, test_size=0.2)
    lr3 = linear_model.LinearRegression()
    lr3.fit(x_train, y_train)
    temp = np.array(lr3.coef_)
    weight_vector_3 = [x + y for x, y in zip(weight_vector_3, temp)]
    predictions_3 = lr3.predict(x_test)
    pcc_3 = stat.pearsonr(predictions_3, y_test)
    counter+=1
    sum += pcc_3[0]

avg_assoc_3 = sum/total_run
avg_weight_vector_3 = np.divide(weight_vector_3, total_run)
print(avg_assoc_3)
print(avg_weight_vector_3)
top2000_index_3 = (-avg_weight_vector_3).argsort()[:2000]
print (top2000_index_3)
print (avg_weight_vector_3[top2000_index_3])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_4 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPDSDS, test_size=0.2)
    lr4 = linear_model.LinearRegression()
    lr4.fit(x_train, y_train)
    temp = np.array(lr4.coef_)
    weight_vector_4 = [x + y for x, y in zip(weight_vector_4, temp)]
    predictions_4 = lr4.predict(x_test)
    pcc_4 = stat.pearsonr(predictions_4, y_test)
    counter+=1
    sum += pcc_4[0]

avg_assoc_4 = sum/total_run
avg_weight_vector_4 = np.divide(weight_vector_4, total_run)
print(avg_assoc_4)
print(avg_weight_vector_4)
top2000_index_4 = (-avg_weight_vector_4).argsort()[:2000]
print (top2000_index_4)
print (avg_weight_vector_4[top2000_index_4])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_5 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGALACTOSE, test_size=0.2)
    lr5 = linear_model.LinearRegression()
    lr5.fit(x_train, y_train)
    temp = np.array(lr5.coef_)
    weight_vector_5 = [x + y for x, y in zip(weight_vector_5, temp)]
    predictions_5 = lr5.predict(x_test)
    pcc_5 = stat.pearsonr(predictions_5, y_test)
    counter+=1
    sum += pcc_5[0]

avg_assoc_5 = sum/total_run
avg_weight_vector_5 = np.divide(weight_vector_5, total_run)
print(avg_assoc_5)
print(avg_weight_vector_5)
top2000_index_5 = (-avg_weight_vector_5).argsort()[:2000]
print (top2000_index_5)
print (avg_weight_vector_5[top2000_index_5])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_6 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPGLYCEROL, test_size=0.2)
    lr6 = linear_model.LinearRegression()
    lr6.fit(x_train, y_train)
    temp = np.array(lr6.coef_)
    weight_vector_6 = [x + y for x, y in zip(weight_vector_6, temp)]
    predictions_6 = lr6.predict(x_test)
    pcc_6 = stat.pearsonr(predictions_6, y_test)
    counter+=1
    sum += pcc_6[0]

avg_assoc_6 = sum/total_run
avg_weight_vector_6 = np.divide(weight_vector_6, total_run)
print(avg_assoc_6)
print(avg_weight_vector_6)
top2000_index_6 = (-avg_weight_vector_6).argsort()[:2000]
print (top2000_index_6)
print (avg_weight_vector_6[top2000_index_6])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_7 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPRIBOSE, test_size=0.2)
    lr7 = linear_model.LinearRegression()
    lr7.fit(x_train, y_train)
    temp = np.array(lr7.coef_)
    weight_vector_7 = [x + y for x, y in zip(weight_vector_7, temp)]
    predictions_7 = lr7.predict(x_test)
    pcc_7 = stat.pearsonr(predictions_7, y_test)
    counter+=1
    sum += pcc_7[0]

avg_assoc_7 = sum/total_run
avg_weight_vector_7 = np.divide(weight_vector_7, total_run)
print(avg_assoc_7)
print(avg_weight_vector_7)
top2000_index_7 = (-avg_weight_vector_7).argsort()[:2000]
print (top2000_index_7)
print (avg_weight_vector_7[top2000_index_7])
#-------------------------------------------------------------------------#


sum = 0
weight_vector_8 =np.zeros(18931)
for counter in range (total_run):
    x_train,x_test,y_train,y_test = train_test_split(snp_yeast_matrix, YPSORBITOL, test_size=0.2)
    lr8 = linear_model.LinearRegression()
    lr8.fit(x_train, y_train)
    temp = np.array(lr8.coef_)
    weight_vector_8 = [x + y for x, y in zip(weight_vector_8, temp)]
    predictions_8 = lr8.predict(x_test)
    pcc_8 = stat.pearsonr(predictions_8, y_test)
    counter+=1
    sum += pcc_8[0]

avg_assoc_8 = sum/total_run
avg_weight_vector_8 = np.divide(weight_vector_8, total_run)
print(avg_assoc_8)
print(avg_weight_vector_8)
top2000_index_8 = (-avg_weight_vector_8).argsort()[:2000]
print (top2000_index_8)
print (avg_weight_vector_8[top2000_index_8])

0.023049784781522597
[-1.25151737e+07 -2.39083391e+06 -1.40556234e+08 ... -1.83953816e+05
  1.80363830e+06  9.36414030e+04]
[    5     8    10 ... 12940 14985 17163]
[53692413.37168993 50486468.71028099 31556678.27359788 ...
   987760.49422795   987738.02277059   987584.7823681 ]
0.011905312874133983
[-37795819.86603655  25954565.06992191  24325450.72223711 ...
     54768.8759803    -715201.39416781   -734297.03116277]
[    1     2     4 ...  7881  3404 17768]
[25954565.06992191 24325450.72223711 23482493.03957968 ...
   403381.95885093   403122.70700828   402853.66336425]
0.03288680044296333
[-62683051.5268332   -9418139.95313801 -17423621.62715344 ...
    553560.87029667    765128.29515293  -2000518.57550295]
[   5    8   14 ... 2229 5776 5775]
[38823308.82927296 23703471.47566    16405154.44709082 ...
  1340373.06389105  1339163.24945174  1339163.24945174]
0.04491667909159437
[-78873242.70871855  44920560.24729865  -3582582.00165882 ...
    160241.72898994   -129347.50798145  -17434

In [6]:
def get_gene(input):
    words = input.split(';')
    gene_tuple = words[2]
    words = gene_tuple.split('=')
    gene = words[1]
    return gene

def snp_pos_to_gene(pos):
    index = bs.bisect(start_pos, pos)
    curr_start = start_pos[index-1]
    curr_tuple = dict[curr_start]
    curr_end = curr_tuple[2]
    if pos in range ((int)(curr_start), (int)(curr_end)):
        return curr_tuple[0]

names=["Contig", "Seq","Type", "Start", "End", "N/A", "N/A1", "N/A2", "Info"]
gene_matrix = pd.read_csv(Gene_url, sep = '\t', error_bad_lines=False, skiprows = 8, names = names)
gene_matrix.head()

dict = {}
gene_list = []
for row in gene_matrix.itertuples():
    if(row.Type == "gene"):
        curr_info = row.Info
        curr_gene = get_gene(curr_info)
        gene_list.append(curr_gene)
        curr_start = row.Start
        curr_end = row.End
        tuple = (curr_gene, curr_start, curr_end)
        dict[curr_start] = tuple
print("Dict finished. There are %u genes" %len(dict))
start_pos = list(dict.keys())
rows = snp_yeast_matrix.index.values
rows = rows[1:]
array = np.zeros([len(rows),len(gene_list)])
yeast_gene_matrix = pd.DataFrame(array, columns = gene_list, index = rows)

yeast_gene_matrix.head()

Dict finished. There are 6376 genes


,PAU8,YAL067W-A,SEO1,YAL065C,YAL064W-B,TDA8,YAL064W,YAL063C-A,FLO9,GDH3,...,tN(GUU)Q,tM(CAU)Q1,COX2,Q0255,tF(GAA)Q,tT(UAG)Q2,tV(UAC)Q,COX3,tM(CAU)Q2,RPM1
AAB_AAB,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAC_AAC,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAD_AAD,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAE_AAE,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
AAG_AAG,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
